In [2]:
!apt-get install default-jre
!java -version
!pip install h2o


Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
     |████████████████████████████████| 164.6MB 97kB/s 
  Created wheel for h2o: filename=h2o-3.32.0.4-py2.py3-none-any.whl size=164670979 sha256=1bbc0a08bd26957d5d056ab1c805f76df4cba1077ea485280955d63bed9f243f
  Stored in directory: /root/.cache/pip/wheels/65/f4/0d/c9bb958d70c2e015c968cb91cbd7f1b486933056d422337d75
Successfully built h2o


In [3]:
import pandas as pd
import numpy as np 
import spacy 
import unicodedata

from spacy.lang.en.stop_words import STOP_WORDS as stopwords
import re
from textblob import TextBlob

from wordcloud import WordCloud 
import matplotlib.pyplot as plt 
%matplotlib inline

import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

import h2o
from h2o.automl import H2OAutoML
h2o.init()



Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpwzxgmnq0
  JVM stdout: /tmp/tmpwzxgmnq0/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpwzxgmnq0/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_unknownUser_my6wa0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Loading Data

In [5]:
data = pd.read_excel('/content/drive/MyDrive/DataSets/QLC /Data1.xlsx',nrows=50000)
data = data.dropna()
data.head()

,Unnamed: 0,OpportunityId,OpportunityStageName,ActivityDescription
0,0,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#ProformaInvoice\r\nDear Sir,\r\n\r\nThanks fo..."
1,1,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#Emailed\r\nDear Sir,\r\n\r\nPlease find revis..."
2,2,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
3,3,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
4,4,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,#Follow up\r\n\r\nCalled on landline number\r\...


Converting it to Dictionary

In [6]:
#convert to dictionary
cov_dic = {'OpportunityId': str,
           'OpportunityStageName': str,
           'ActivityDescription': str}
df= data.astype(cov_dic)
df=df.drop(columns=['Unnamed: 0'])
df.head()

,OpportunityId,OpportunityStageName,ActivityDescription
0,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#ProformaInvoice\r\nDear Sir,\r\n\r\nThanks fo..."
1,0006255A-A476-4DEF-9C67-0E4B0A83ACD0,Closed Won,"#Emailed\r\nDear Sir,\r\n\r\nPlease find revis..."
2,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
3,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,"#Proposal\r\n\r\nDear Ma'am,\r\n\r\nGreetings ..."
4,000992B7-C04E-464A-A010-A2F3F9DCEF8B,Proposal,#Follow up\r\n\r\nCalled on landline number\r\...


Upper case Count

In [7]:
#removing '-' in id
df['OpportunityId'] = df['OpportunityId'].apply(lambda x: str(x).replace('-', ''))

#Concatanating Conversations of each OpportunityID
df = df.groupby(['OpportunityId', 'OpportunityStageName'], as_index = False).agg({'ActivityDescription': ' '.join})
#upper Case
df['upper_count'] = df['ActivityDescription'].apply(lambda x: len([t for t in x.split() if t.isupper()]))

# Lower case
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x:str(x).lower())

#removing accented Chars
def remove_accented_chars(x):
  x= unicodedata.normalize('NFKD',x).encode('ascii','ignore').decode('utf-8','ignore')
  return x
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: remove_accented_chars(x))

#removing '/r' and '/n
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: str(x).replace('\r', ' '))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: str(x).replace('\r\n', ' '))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: str(x).replace('\n', ' '))

#count and remove URLS
df['URLs_count'] = df['ActivityDescription'].apply(lambda x: len(re.findall(r'(http|https|ftp|ssh|www)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^+%&/~+#-])?',str(x))))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub(r'(http|https|ftp|ssh|www)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^+%&/~+#-])?','',str(x)))
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub('(?:\s)www[^, ]*', '', str(x)))

#remove single letters
df['ActivityDescription'] = df.ActivityDescription.str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
# remove all numbers
df['ActivityDescription'] = df.ActivityDescription.str.replace('\d+', '')

df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub(r'[^\w ]+',"",x)) #removing special characters
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: ' '.join(x.split())) #removing double spaces

#counting hashtags
df['hash_count'] = df['ActivityDescription'].apply(lambda x : re.findall(r'(#[Aa])', x))
df['hash_count'] =df['hash_count'].apply(lambda x : len(x))

#count email
df['e_count'] = df['ActivityDescription'].apply(lambda x : re.findall(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+",x))
df['e_count'] = df['e_count'].apply(lambda x:len(x))
#remove email 
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x: re.sub("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+","",str(x)))

#stop word count
df['sw_count']= df['ActivityDescription'].apply(lambda x: len([t for t in x.split() if t in stopwords]))
# removing stop words
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x : ' '.join([t for t in x.split() if t not in stopwords]))

In [8]:
df['URLs_count'].value_counts()

0     4122
2       91
4       80
3       51
1       50
8       12
6        9
5        6
16       2
12       2
15       1
10       1
21       1
Name: URLs_count, dtype: int64

In [9]:
#removing top 20 frequet words
text = ' '.join(df['ActivityDescription'])
text = text.split()
freq_comm = pd.Series(text).value_counts()
f20 = freq_comm[:20]
f20

df['ActivityDescription']= df['ActivityDescription'].apply(lambda x : ' '.join([t for t in x.split() if t not in f20]))


In [10]:
#Label encoding target variable
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['OpportunityStageName']= label_encoder.fit_transform(df['OpportunityStageName']) 

In [11]:
#tokenization
from nltk.tokenize import sent_tokenize, word_tokenize 
#df['ActivityDescription']= df['ActivityDescription'].apply(lambda x: [word for word in word_tokenize(x)])
#word_tokenize(text)
df['ActivityDescription']= df['ActivityDescription'].apply(word_tokenize)

In [12]:
stemmer = nltk.stem.SnowballStemmer('english')
def tokenize(text):
   tokens = text
   stems = [stemmer.stem(item) for item in tokens if (item not in stopwords)]
   return stems

In [13]:
df['ActivityDescription'] = df['ActivityDescription'].apply(lambda x : tokenize(x))

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size=0.2, random_state=111)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words=None, max_df=0.75, max_features=10000, lowercase=False, ngram_range=(1,2))
train_vectors = vectorizer_tf.fit_transform(X_train.ActivityDescription)

In [16]:
test_vectors = vectorizer_tf.transform(X_test.ActivityDescription)

In [17]:
train_vectors

<3542x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 333656 stored elements in Compressed Sparse Row format>

In [18]:
vectorizer_tf.get_feature_names

<bound method CountVectorizer.get_feature_names of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=0.75,
                max_features=10000, min_df=1, ngram_range=(1, 2), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x7f824168bb90>, use_idf=True,
                vocabulary=None)>

In [19]:
X_train.head()

,OpportunityId,OpportunityStageName,ActivityDescription,upper_count,URLs_count,hash_count,e_count,sw_count
2790,275DD4811AAC4805BAC710C2F4E928ED,0,"[stop, app, find, issu, ask, provid, train, ok...",1,0,0,0,39
1285,11A2B77CF1C44B3EA50F7C68358AFFF7,2,"[detail, enter, slam, want, today, pm, user, e...",12,2,0,0,90
3848,35D86DBB75F5494F818BC03F54599381,3,"[firstcontactestablish, total, present, increa...",52,0,0,0,400
4216,3AA237EE4AA24C23A8C41C89944523C0,0,"[new, entri, ad, slam, spoke, smirti, user, re...",41,2,0,0,254
3889,366561B57C9E420486D5B41DCC2E9A29,0,"[emails, greet, qlc, request, provid, skype, i...",2,0,0,0,36


In [20]:
train_df=pd.DataFrame(train_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
train_df=pd.concat([train_df,X_train['OpportunityStageName'].reset_index(drop=True)], axis=1)
train_df=pd.concat([train_df,X_train[['upper_count','hash_count','e_count','sw_count']].reset_index(drop=True)], axis=1)
train_df

,_________________________________________________________________,_________________________________________________________________ invoic,________________________________________________________________________,________________________________________________________________________ invoic,________________________________________________________________________________,________________________________________________________________________________ invoic,___________________________________________________________________________________,___________________________________________________________________________________ invoic,_spoke,aa,aa video,aaacqn,aaacqn id,aaacqn servic,aaacqnzg,aaacqnzg id,aabcbl,aabcblz,aarti,aarti busi,abck,abdulkarimov,abhay,abhijeet,abhishek,abl,abl add,abl check,abl confirm,abl connect,abl login,abl meet,abl pay,abl provid,abl reach,abl real,abl relea,abl send,abl start,abl track,...,yesterday,yesterday discuss,yesterday log,yesterday ms,yogendra,yogesh,yogesh inform,youad,youall,youall receiv,youar,youar experi,youar minut,youav,youav ad,youll,youll consid,youtub,youtub channel,yr,yr contract,yr free,yrs,yrs addit,yrs contract,yrs free,yrs valid,yrs yrs,yusuf,zankhana,zimbra,zoho,zone,zoom,zoom meet,OpportunityStageName,upper_count,hash_count,e_count,sw_count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,39
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,12,0,0,90
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,52,0,0,400
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,41,0,0,254
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2,0,0,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,31
3538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,20,0,0,67
3539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [21]:
test_df=pd.DataFrame(test_vectors.toarray(), columns=vectorizer_tf.get_feature_names())
test_df=pd.concat([test_df,X_test['OpportunityStageName'].reset_index(drop=True)], axis=1)
test_df=pd.concat([test_df,X_test[['upper_count','hash_count','e_count','sw_count']].reset_index(drop=True)], axis=1)
test_df

,_________________________________________________________________,_________________________________________________________________ invoic,________________________________________________________________________,________________________________________________________________________ invoic,________________________________________________________________________________,________________________________________________________________________________ invoic,___________________________________________________________________________________,___________________________________________________________________________________ invoic,_spoke,aa,aa video,aaacqn,aaacqn id,aaacqn servic,aaacqnzg,aaacqnzg id,aabcbl,aabcblz,aarti,aarti busi,abck,abdulkarimov,abhay,abhijeet,abhishek,abl,abl add,abl check,abl confirm,abl connect,abl login,abl meet,abl pay,abl provid,abl reach,abl real,abl relea,abl send,abl start,abl track,...,yesterday,yesterday discuss,yesterday log,yesterday ms,yogendra,yogesh,yogesh inform,youad,youall,youall receiv,youar,youar experi,youar minut,youav,youav ad,youll,youll consid,youtub,youtub channel,yr,yr contract,yr free,yrs,yrs addit,yrs contract,yrs free,yrs valid,yrs yrs,yusuf,zankhana,zimbra,zoho,zone,zoom,zoom meet,OpportunityStageName,upper_count,hash_count,e_count,sw_count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.039948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,3,32,0,0,73
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.038663,0.0,0.042441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,3,138,0,0,705
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,1,0,0,134
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,7
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.081911,0.0,5,110,0,0,878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112926,0.114696,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100056,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,4,0,0,85
882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.

## Implementing H2o for model selection

In [22]:
h2o_train_df = h2o.H2OFrame(train_df)
h2o_test_df = h2o.H2OFrame(test_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [23]:
h2o_train_df.types

{'_________________________________________________________________': 'real',
 '_________________________________________________________________ invoic': 'real',
 '________________________________________________________________________': 'real',
 '________________________________________________________________________ invoic': 'real',
 '________________________________________________________________________________': 'real',
 '________________________________________________________________________________ invoic': 'real',
 '___________________________________________________________________________________': 'real',
 '___________________________________________________________________________________ invoic': 'real',
 '_spoke': 'real',
 'aa': 'real',
 'aa video': 'real',
 'aaacqn': 'real',
 'aaacqn id': 'real',
 'aaacqn servic': 'real',
 'aaacqnzg': 'real',
 'aaacqnzg id': 'real',
 'aabcbl': 'real',
 'aabcblz': 'real',
 'aarti': 'real',
 'aarti busi': 'real',
 'abck': 'real',
 'ab

In [24]:
h2o_train_df['OpportunityStageName'] = h2o_train_df['OpportunityStageName'].asfactor()
h2o_test_df['OpportunityStageName'] = h2o_test_df['OpportunityStageName'].asfactor()

In [25]:
aml = H2OAutoML(max_models = 5, seed = 10, exclude_algos = ["StackedEnsemble"], verbosity="info", nfolds=0, balance_classes=True, max_after_balance_size=0.3)

here i have set it to use top 10 models



In [26]:
x=vectorizer_tf.get_feature_names()
y='OpportunityStageName'

In [27]:
aml.train(x = x, y = y, training_frame = h2o_train_df, validation_frame=h2o_test_df)

AutoML progress: |
09:56:22.271: Project: AutoML_20210227_95622267
09:56:22.274: Cross-validation disabled by user: no fold column nor nfolds > 1.
09:56:22.304: Setting stopping tolerance adaptively based on the training frame: 0.01680257031774874
09:56:22.304: Build control seed: 10
09:56:22.305: Since cross-validation is disabled, and no leaderboard frame was provided, automatically split the training data into training and leaderboard frames in the ratio 90/10
09:56:45.59: training frame: Frame key: automl_training_py_1_sid_894e    cols: 10005    rows: 3182  chunks: 36    size: 32486240  checksum: 4506362551962613624
09:56:45.107: validation frame: Frame key: py_2_sid_894e    cols: 10005    rows: 886  chunks: 9    size: 8223463  checksum: 404194905631009924
09:56:47.969: leaderboard frame: Frame key: automl_leaderboard_py_1_sid_894e    cols: 10005    rows: 360  chunks: 35    size: 28565120  checksum: -380945111892243190
09:56:47.969: blending frame: NULL
09:56:47.969: response colum

Works best with XGBoost 

In [28]:
aml.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse,auc,aucpr
XGBoost_2_AutoML_20210227_095622,0.430256,0.756948,0.464452,0.215716,nan,nan
XGBoost_3_AutoML_20210227_095622,0.431631,0.703746,0.457511,0.209316,nan,nan
GLM_1_AutoML_20210227_095622,0.434123,0.772186,0.484381,0.234625,nan,nan
XGBoost_1_AutoML_20210227_095622,0.439647,0.758215,0.464654,0.215903,nan,nan
DRF_1_AutoML_20210227_095622,0.457985,0.835771,0.504976,0.255001,nan,nan


In [29]:
pred=aml.leader.predict(h2o_test_df)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [30]:
aml.leader.model_performance(h2o_test_df)


ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 0.2196562629265232
RMSE: 0.4686750077895377
LogLoss: 0.7884869457012021
Mean Per-Class Error: 0.5844390786216951
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,Error,Rate
0,470.0,0.0,1.0,33.0,0.0,0.0,0.0,0.0,10.0,16.0,0.0,0.113208,60 / 530
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,4 / 4
2,19.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.952381,20 / 21
3,40.0,0.0,0.0,134.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.247191,44 / 178
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.000000,4 / 4
5,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,4 / 4
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0 / 0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0 / 0
8,36.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.725806,45 / 62
9,31.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.390244,32 / 82



Top-10 Hit Ratios: 


,k,hit_ratio
0,1,0.758465
1,2,0.899549
2,3,0.952596
3,4,0.980813
4,5,0.989842
5,6,0.992099
6,7,0.993228
7,8,0.995485
8,9,1.000000
9,10,1.000000


In [31]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
out = h2o.get_model([mid for mid in model_ids if "XGBoost" in mid][0])
out

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_2_AutoML_20210227_095622


Model Summary: 


,,number_of_trees
0,,45.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.0821393029438277
RMSE: 0.28659955154156763
LogLoss: 0.301293713604789
Mean Per-Class Error: 0.629265982321738
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,Error,Rate
0,1971.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,5.0,7.0,0.0,0.011535,"23 / 1,994"
1,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,12 / 12
2,41.0,0.0,58.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.457944,49 / 107
3,50.0,0.0,0.0,554.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.084298,51 / 605
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,1.000000,8 / 8
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,4 / 4
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1 / 1
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,2 / 2
8,42.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,143.0,1.0,0.0,0.262887,51 / 194
9,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,221.0,0.0,0.105263,26 / 247



Top-10 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc,validation_rmse,validation_logloss,validation_classification_error,validation_auc,validation_pr_auc
0,,2021-02-27 09:57:47,0.430 sec,0.0,0.909091,2.397895,0.809868,NaN,NaN,0.909091,2.397895,0.799097,NaN,NaN
1,,2021-02-27 09:57:59,12.471 sec,5.0,0.557859,0.905749,0.224073,NaN,NaN,0.595630,1.043607,0.279910,NaN,NaN
2,,2021-02-27 09:58:02,15.652 sec,10.0,0.450550,0.640262,0.193589,NaN,NaN,0.516679,0.855402,0.262980,NaN,NaN
3,,2021-02-27 09:58:06,19.988 sec,15.0,0.398753,0.516544,0.158077,NaN,NaN,0.492139,0.804490,0.261851,NaN,NaN
4,,2021-02-27 09:58:10,23.193 sec,20.0,0.366563,0.447824,0.134507,NaN,NaN,0.480393,0.782302,0.256208,NaN,NaN
5,,2021-02-27 09:58:13,26.710 sec,25.0,0.342897,0.401727,0.112194,NaN,NaN,0.475569,0.779584,0.251693,NaN,NaN
6,,2021-02-27 09:58:16,29.344 sec,30.0,0.324343,0.368100,0.097423,NaN,NaN,0.473454,0.780244,0.242664,NaN,NaN
7,,2021-02-27 09:58:20,33.758 sec,35.0,0.308986,0.341117,0.086109,NaN,NaN,0.470423,0.779683,0.246050,NaN,NaN
8,,2021-02-27 09:58:23,36.561 sec,40.0,0.297762,0.318867,0.080767,NaN,NaN,0.468661,0.782605,0.243792,NaN,NaN
9,,2021-02-27 09:58:27,40.019 sec,45.0,0.286600,0.301294,0.073853,NaN,NaN,0.468675,0.788487,0.241535,NaN,NaN



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,invoic,950.908081,1.000000,0.105588
1,bank,371.819000,0.391015,0.041286
2,notrespond,277.177032,0.291487,0.030777
3,entri,251.961548,0.264969,0.027978
4,slam entri,244.225800,0.256834,0.027119
5,requir,221.099640,0.232514,0.024551
6,proforma,148.865051,0.156550,0.016530
7,pme,119.115929,0.125265,0.013227
8,pvt ltd,116.896835,0.122932,0.012980
9,process,116.524185,0.122540,0.012939



See the whole table with table.as_data_frame()


In [32]:
out.convert_H2OXGBoostParams_2_XGBoostParams()

({'alpha': 0.0,
  'booster': 'gbtree',
  'colsample_bylevel': 0.8,
  'colsample_bytree': 0.8,
  'eta': 0.3,
  'gamma': 0.0,
  'grow_policy': 'depthwise',
  'lambda': 1.0,
  'max_delta_step': 0.0,
  'max_depth': 20,
  'min_child_weight': 10.0,
  'nround': 10000,
  'nthread': 2,
  'num_class': 11,
  'objective': 'multi:softprob',
  'seed': 11,
  'silent': True,
  'subsample': 0.6,
  'tree_method': 'exact'},
 10000)

In [33]:

xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, objective='multi:softprob', random_state=10, **{"updater": "grow_gpu"})

In [34]:

xgb_clf.fit(train_vectors, X_train['OpportunityStageName'])

[10:43:32] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, updater='grow_gpu', verbosity=1)

In [35]:
predictions = xgb_clf.predict(test_vectors)

In [36]:
cm = confusion_matrix(X_test['OpportunityStageName'], predictions)
print(cm)

[[471   0   0  33   0   0   9  17   0]
 [  3   0   0   0   0   0   0   1   0]
 [ 20   0   0   1   0   0   0   0   0]
 [ 42   0   1 129   0   0   4   2   0]
 [  3   0   0   0   0   0   0   1   0]
 [  2   0   0   1   0   0   0   1   0]
 [ 44   0   0  10   0   0   8   0   0]
 [ 39   1   0   1   0   0   1  40   0]
 [  0   0   0   1   0   0   0   0   0]]


In [37]:
print('classification_report :\n',classification_report(X_test['OpportunityStageName'], predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification_report :
               precision    recall  f1-score   support

           0       0.75      0.89      0.82       530
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00        21
           3       0.73      0.72      0.73       178
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         4
           8       0.36      0.13      0.19        62
           9       0.65      0.49      0.56        82
          10       0.00      0.00      0.00         1

    accuracy                           0.73       886
   macro avg       0.28      0.25      0.25       886
weighted avg       0.68      0.73      0.70       886



## Class Imbalance

In [38]:
X_train['OpportunityStageName'].value_counts()

0     2200
3      689
9      284
8      216
2      116
1       13
4        9
10       8
5        4
7        2
6        1
Name: OpportunityStageName, dtype: int64

In [39]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight('balanced',
                                             np.unique(X_train['OpportunityStageName']),
                                             X_train['OpportunityStageName']))

In [40]:
class_weights

[0.14636363636363636,
 24.76923076923077,
 2.7758620689655173,
 0.46734397677793904,
 35.77777777777778,
 80.5,
 322.0,
 161.0,
 1.4907407407407407,
 1.1338028169014085,
 40.25]

In [41]:
weights = np.ones(X_train.shape[0], dtype = 'float')
weights

array([1., 1., 1., ..., 1., 1., 1.])

In [42]:
for i, val in enumerate(X_train['OpportunityStageName']):
    weights[i] = class_weights[val]

weights

array([0.14636364, 2.77586207, 0.46734398, ..., 0.14636364, 1.49074074,
       0.14636364])

In [43]:
xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, objective='multi:softprob', random_state=10, **{"updater": "grow_gpu"})

In [44]:
xgb_clf.fit(train_vectors, X_train['OpportunityStageName'], sample_weight=weights)

[10:43:46] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, updater='grow_gpu', verbosity=1)

In [45]:
predictions = xgb_clf.predict(test_vectors)

In [46]:
cm = confusion_matrix(X_test['OpportunityStageName'], predictions)
print(cm)

[[323   0  18  53   0   5   0   0  72  58   1]
 [  2   1   0   0   0   0   0   0   0   1   0]
 [ 13   0   4   1   0   0   0   0   2   1   0]
 [  6   0   5 142   0   4   1   0  18   1   1]
 [  0   0   0   0   0   0   0   0   0   4   0]
 [  2   0   0   1   0   0   0   0   0   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0]
 [ 15   0   3  11   0   1   0   1  31   0   0]
 [ 19   1   1   2   0   1   0   0   0  57   1]
 [  0   0   0   1   0   0   0   0   0   0   0]]


In [47]:
print('classification_report :\n',classification_report(X_test['OpportunityStageName'], predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification_report :
               precision    recall  f1-score   support

           0       0.85      0.61      0.71       530
           1       0.50      0.25      0.33         4
           2       0.13      0.19      0.15        21
           3       0.67      0.80      0.73       178
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.25      0.50      0.34        62
           9       0.46      0.70      0.56        82
          10       0.00      0.00      0.00         1

    accuracy                           0.63       886
   macro avg       0.26      0.28      0.26       886
weighted avg       0.71      0.63      0.65       886

